# 第五課 - Teachable machine  
請先到Teachable machine網站訓練一個自己的動物辨認模型:  
https://teachablemachine.withgoogle.com/  
  
或是下載本課程已訓練好的貓狗分辨模型:  
https://github.com/LBK888/ComVis/tree/main

In [ ]:
#!pip install tensorflow==2.12.1   #restart session after this

from IPython import get_ipython
from IPython.display import display
# %%
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import requests
from io import BytesIO
import random

'''
import h5py

f = h5py.File("keras_model.h5", mode="r+")
model_config_string = f.attrs.get("model_config")

if model_config_string.find('"groups": 1,') != -1:
    model_config_string = model_config_string.replace('"groups": 1,', '')
f.attrs.modify('model_config', model_config_string)
f.flush()

model_config_string = f.attrs.get("model_config")

assert model_config_string.find('"groups": 1,') == -1
'''


# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("/content/keras_model.h5", compile=False)

# Load the labels
class_names = open("/content/labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Function to fetch a random cat or dog image
def get_random_pet_image():
    # api doc: https://api.sefinek.net/docs/v2

    pet_type = random.choice(["dog", "cat", "fish", "bird"])
    url = f"https://api.sefinek.net/api/v2/random/animal/{pet_type}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad responses

    # Extract image URL from JSON response
    image_url = response.json()['message']

    # Fetch the image using the extracted URL
    image_response = requests.get(image_url, stream=True)
    image_response.raise_for_status()
    image = Image.open(BytesIO(image_response.content)).convert("RGB")
    return image, pet_type


Accuracy=0
repeat=10
for i in range(repeat):
  # Get random image
  image, pet_type = get_random_pet_image()

  # Preprocess the image
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
  image_array = np.asarray(image)
  normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
  data[0] = normalized_image_array

  # Display the image
  display(image)

  # Predict using the model
  prediction = model.predict(data)
  index = np.argmax(prediction)
  class_name = class_names[index]
  confidence_score = prediction[0][index]

  # Print prediction and confidence score
  print("Class:", class_name[2:], end="")
  print("Confidence Score:", confidence_score)

  if pet_type==class_name[2:-1]:
    Accuracy+=1

print("Accuracy:", Accuracy*100/repeat,"%")
